In [ ]:
from icecream import ic
import numpy as np
import rp
import torch
import torch.nn as nn
import source.stable_diffusion as sd
from source.clip import get_clip_logits
from source.learnable_textures import (LearnableImageFourier,
                                       LearnableImageFourierBilateral,
                                       LearnableImageRaster,
                                       LearnableImageRasterBilateral,
                                       LearnableTexturePackFourier,
                                       LearnableTexturePackRaster,
                                       LearnableImageRasterBoxBlurredSigmoided,
                                      )
from source.stable_diffusion_labels import BaseLabel, NegativeLabel, SimpleLabel

In [ ]:
if 's' not in dir():
    model_name="CompVis/stable-diffusion-v1-4"
    # model_name="nitrosocke/nitro-diffusion"
    model_name="stabilityai/stable-diffusion-2-1"
    gpu=rp.select_torch_device()
    s=sd.StableDiffusion(gpu,model_name)
device=s.device

In [ ]:
rp.pretty_print(dict(s.scheduler.config))
print(s.scheduler.prediction_type)
# s.scheduler.config.prediction_type='epsilon'
# s.scheduler.config.prediction_type='epsilon'
# s.scheduler.config.prediction_type='v_prediction'

In [ ]:
#ONLY GOOD PROMPTS HERE
# prompt = 'A cute kitten in a cardboard box in times square'
# prompt = 'The Legend of Zelda landscape atmospheric, hyper realistic, 8k, epic composition, cinematic, octane render, artstation landscape vista photography by Carr Clifton & Galen Rowell, 16K resolution, Landscape veduta photo by Dustin Lefevre & tdraw, 8k resolution, detailed landscape painting by Ivan Shishkin, DeviantArt, Flickr, rendered in Enscape, Miyazaki, Nausicaa Ghibli, Breath of The Wild, 4k detailed post processing, artstation, rendering by octane, unreal engine —ar 16:9'
# prompt = 'ultra realistic photo portrait of Scarlett Leithold cosmic energy, colorful, painting burst, beautiful symmetrical face, nonchalant kind look, realistic round eyes, tone mapped, intricate, elegant, highly detailed, digital painting, artstation, concept art, smooth, sharp focus, illustration, dreamy magical atmosphere, art by artgerm and greg rutkowski and alphonse mucha, 4k, 8k'
# prompt = 'Insanely detailed studio portrait shot photo of intricately detailed beautiful yorkshire terrier dressed as santa clause, smirking mischievously at the camera with mischievous detailed yellow green eyes , very detailed, rim light, photo, rim light, ultra-realistic, photorealistic, hyper detailed, photography, shot on Canon DSLR, f/2. 8 , photography by Felix Kunze and Annie Leibovitz and retouched by Pratik Naik'
# prompt = 'professional portrait photograph of a gorgeous Norwegian girl in winter clothing with long wavy blonde hair, freckles, gorgeous symmetrical face, cute natural makeup, wearing elegant warm winter fashion clothing, ((standing outside in snowy city street)), mid shot, central image composition, (((professionally color graded))), (((bright soft diffused light)))'
# prompt = '8 k concept art from a hindu temple lost in the jungle by david mattingly and samuel araya and michael whelan and dave mckean and richard corben. realistic matte painting with photorealistic hdr volumetric lighting. composition and layout inspired by gregory crewdson. '
# prompt = 'a big sailing ship in heavy sea, hypermaximalistic, high details, cinematic, 8k resolution, beautiful detailed, insanely intricate details, artstation trending, octane render, unreal engine'
# prompt = 'giant standalone lighthouse from bioshock infinite in england 1 9 century, half - ruined, covered by mold, staying in 2 kilometers far from a coast, opposite the dark cave - crack of giant rocks. when you see this lighthouse it makes you anxious. deep ones is living under this. view from sea, and view from the coast, by greg rutkowski'
# prompt = 'photo of bunny hugging another bunny, dramatic light, pale sunrise, cinematic lighting'
# prompt = 'thomas the tank engine as a military tank, intricate, highly detailed, centered, digital painting, artstation, concept art, smooth, sharp focus, illustration, artgerm, tomasz alen kopera, peter mohrbacher, donato giancola, joseph christian leyendecker, wlop, boris vallejo'
# prompt = 'a wolf with a tail, standing heroically on a rock. adventurous, new adventure, with a tail, forest, rocks, stream, ripples, tribal armor, female, wolf wolf wolf, atmospheric lighting, stunning, brave. by makoto shinkai, stanley artgerm lau, wlop, rossdraws, james jean, andrei riabovitchev, marc simonetti, krenz cushart, sakimichan, d & d trending on artstation, digital art. '
prompt = 'lolita dress, angelic pretty, award winning photograph trending on artstation'
# prompt = 'lolita dress, angelic pretty, portrait of magical lolita woman elf elven,  hyperrealism photography hdr 4k 3d, dreamy and ethereal, fantasy, intricate, elegant, many rainbow bubbles, rose tones, highly detailed, artstation, concept art, cyberpunk wearing, smooth, sharp focus, illustration, art by artgerm and greg rutkowskiand alphonse mucha'
# prompt = 'an intricate detailed hb pencil sketch of a giraffe head'
# prompt = 'an intricate detailed hb pencil sketch of a penguin'
# prompt = 'an intricate detailed hb pencil sketch of a violin'
# prompt = 'an orca whale spouting water intricate detailed hb pencil sketch of an black white spotted orca whale'
# prompt = 'Hollow knight by ari gibson, album cover'
# prompt = 'arcane style, jinx'
# prompt = 'an intricate detailed hb pencil sketch of a black white spotted cow'
# prompt = 'an intricate detailed hb pencil sketch of a walrus'
# prompt = 'an sketch of a cat head'
# prompt = 'a sketch of a penguin'
label = SimpleLabel(prompt)

In [ ]:
for _ in range(1):
    rp.display_image(label.get_sample_image())

In [ ]:
#Pure Noise img 3

#Random timestep vs this scheduleShould be faster.

#Select Learnable Image Type:
# image_device=rp.select_torch_device()
# learnable_image = LearnableImageFourier().to(s.device)
# learnable_image = LearnableImageFourier(height=384,width=384,num_features=512,hidden_dim=512,scale=20).to(s.device)
# learnable_image = LearnableImageFourier(height=384,width=384,num_features=256,hidden_dim=256,scale=20).to(s.device)
# learnable_image = LearnableImageFourier(height=512,width=512,num_features=512,hidden_dim=512,scale=20).to(s.device)
lr=1e-4 #For fourier


# learnable_image = LearnableImageRaster(height=512,width=512).to(device)
learnable_image = LearnableImageRasterBoxBlurredSigmoided(height=s.height,width=s.width).to(device)
lr=1e1 #For raster



optim=torch.optim.SGD(learnable_image.parameters(),lr=lr)

NUM_ITER=1000
s.max_step=MAX_STEP=990
# s.min_step=MIN_STEP=450
s.min_step=MIN_STEP=10

et=rp.eta(NUM_ITER)

folder='untracked/sd_previewer_results3/'+prompt[:100]+rp.random_namespace_hash()
rp.make_folder(folder)
ims=[]

for iter_num in range(NUM_ITER):
    
    step = rp.blend(MAX_STEP,MIN_STEP,iter_num/NUM_ITER)
    s.min_step = s.max_step = int(step)
    
    et(iter_num)

    image=learnable_image()

    pred=s.train_step(
        label.embedding,
        image[None],

        #PRESETS (uncomment one):
        # noise_coef=.1,guidance_scale=50,#10
        # noise_coef=0,image_coef=-.01,guidance_scale=50,
        # noise_coef=0,image_coef=-.005,guidance_scale=50,
        # noise_coef=.1,image_coef=-.010,guidance_scale=50,
        # noise_coef=.1,image_coef=-.005,guidance_scale=50,
        # noise_coef=.1/(s.height**2/512**2), latent_coef=.0, image_coef=0, guidance_scale=50 #WORKS WITH SD2
        # noise_coef=.0, latent_coef=.1/(s.height**2/512**2), image_coef=0, guidance_scale=50,
        # noise_coef=.0, latent_coef=.1/(s.height**2/512**2), image_coef=0, guidance_scale=15,
        noise_coef=.0, latent_coef=0, image_coef=0.13, guidance_scale=15,
        # target=None if not iter_num%10 else target #Target idea! Right now increasing target steps decreases diffusion steps.
        image_target=None if not iter_num%10 else image_target #
    )
    
    # target=pred.target
    image_target=pred.image_target
    ims.append(image)

    with torch.no_grad():
        if not iter_num%50:
            im=rp.as_numpy_image(learnable_image())
            rp.save_image(im,folder+'/%06i.png'%iter_num)
            rp.display_image(im)
        if not iter_num%100:
            gim=rp.tiled_images(rp.as_numpy_images(s.decode_latents(pred.deprecated_output)))
            rp.display_image(gim)
            rp.save_image(gim,folder+'/G%06i.png'%iter_num)

    optim.step()
    optim.zero_grad()


In [ ]:
rp.display_image_slideshow(ims)